In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [115]:
import model
import train
import evaluate
import train_util
import data_util.data
import data_util.batcher
import data_util.config
import data_util.preprocess

importlib.reload(data_util.preprocess)
importlib.reload(train)
importlib.reload(model)
importlib.reload(evaluate)
importlib.reload(train_util)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from train import *
from evaluate import *
from model import *
from train_util import *
from data_util.data import *
from data_util.batcher import *
from data_util.preprocess import *

In [30]:
task_train = pd.read_csv(os.path.join(config.sum_path, 'task_train.tsv'), sep = '\t')
wiki_train = pd.read_csv(os.path.join(config.sum_path, 'wiki_train.tsv'), sep = '\t')
wiki_val   = pd.read_csv(os.path.join(config.sum_path, 'wiki_val.tsv'  ), sep = '\t')
task_val   = pd.read_csv(os.path.join(config.sum_path, 'task_val.tsv'  ), sep = '\t')
task_test  = pd.read_csv(os.path.join(config.sum_path, 'task_test.tsv' ), sep = '\t')

In [31]:
# replace NAs with ''
for df in [task_train, wiki_train, wiki_val, task_val, task_test]:
    df.replace(np.nan, '', inplace=True)

In [32]:
# check sizes
print(task_train.shape)
print(wiki_train.shape)
print(wiki_val.shape)
print(task_val.shape)
print(task_test.shape)

(3235, 3)
(917986, 3)
(300, 3)
(200, 3)
(800, 3)


In [33]:
def wc(df, cols):
    for c in cols:
        df[c + 'WC'] = df[c].map(lambda text: len(text.split()))

In [34]:
for df in [task_train, task_val, task_test]:
    df.drop_duplicates(['Context', 'TaskSentence'], inplace=True)
    wc(df, ['Summary'])

In [35]:
task_train.describe()

,SummaryWC
count,3101.000000
mean,5.168655
std,2.828814
min,1.000000
25%,3.000000
50%,5.000000
75%,6.000000
max,33.000000


In [74]:
# upsample longer summaries
rep = 2
long_sum = task_train['SummaryWC'] > 4
task_train = task_train.append([task_train[long_sum]] * rep, ignore_index=True).sample(frac=1)

Testing TaskExample for wiki

In [99]:
ex = list(wiki_train[6:1000].itertuples())[0]
ts = TaskExample(ex.Context, ex.TaskSentence, ex.Summary, vocab)

CONTEXT: [ , Since, pets, are, considered, property, ,, if, you, can, show, that, you, purchased, the animal, prior, to, the marriage, or, used, non, -, marital, funds, to, buy, the pet, ,, you, may, have, a better chance, of, keeping, the pet, in, the divorce, .] 

TASK:    [However, ,, if, you, did, not, provide, primary care, for, the pet, during, marriage, or, pay, for, its care, ,, you, may, be, required, to, reimburse, you, spouse, for, the cost, of, care, if, you, are, awarded, the pet, .] 

SUMMARY: [determine purchaser]
entity label {}
chunk head {'you': 'you', 'primary care': 'care', 'the pet': 'pet', 'marriage': 'marriage', 'its care': 'care', 'the cost': 'cost', 'care': 'care', 'pets': 'pets', 'the animal': 'animal', 'the marriage': 'marriage', 'a better chance': 'chance', 'the divorce': 'divorce'}
OOVs [',']


In [76]:
ts.target

[0, 3]

In [24]:
    print(ex)

Pandas(Index=6, Summary='determine purchaser', TaskSentence='However, if you did not provide primary care for the pet during marriage or pay for its care, you may be required to reimburse you spouse for the cost of care if you are awarded the pet.', Context=' Since pets are considered property, if you can show that you purchased the animal prior to the marriage or used non-marital funds to buy the pet, you may have a better chance of keeping the pet in the divorce.')


In [112]:
list(nlp('please help the famous Rowan Cassius at Enron Co. fight poverty in the north side').noun_chunks)

[the famous Rowan Cassius, Enron Co., poverty, the north side]

In [116]:
examples = []
for i,e in enumerate(wiki_train[6:200].itertuples()):
    ts = TaskExample(e.Context, e.TaskSentence, e.Summary, vocab)

CONTEXT: [ , Since, pets, are, considered, property, ,, if, you, can, show, that, you, purchased, the animal, prior, to, the marriage, or, used, non, -, marital, funds, to, buy, the pet, ,, you, may, have, a better chance, of, keeping, the pet, in, the divorce, .] 

TASK:    [However, ,, if, you, did, not, provide, primary care, for, the pet, during, marriage, or, pay, for, its care, ,, you, may, be, required, to, reimburse, you, spouse, for, the cost, of, care, if, you, are, awarded, the pet, .] 

SUMMARY: [determine purchaser]
entity label {}
chunk head {'you': 'you', 'primary care': 'care', 'the pet': 'pet', 'marriage': 'marriage', 'its care': 'care', 'the cost': 'cost', 'care': 'care', 'pets': 'pets', 'the animal': 'animal', 'the marriage': 'marriage', 'a better chance': 'chance', 'the divorce': 'divorce'}
OOVs [',']
CONTEXT: [ , When, you, need, to, reduce, blood pressure, ,, you, should, eat, whole grains, ., Instead, of, choosing, refined grains, such, as, white bread, ,, rice, 

entity label {}
chunk head {'you': 'you', 'those effects': 'effects', 'one thing': 'thing', 'mind': 'mind', 'vanish': 'vanish', 'it': 'it', 'bleed or magical effects': 'effects'}
OOVs [',']
CONTEXT: [ A joint venture, can, be, a partnership, ,, whereby, the two original companies, each, retain, their original identity, .] 

TASK:    [The joint venture agreement, becomes, a partnership agreement, that, will, define, how, the partners, will, share, liabilities, and, responsibilities, .] 

SUMMARY: [form, a partnership, if, you, wish, to, retain, individual identities]
entity label {}
chunk head {'the joint venture agreement': 'agreement', 'a partnership agreement': 'agreement', 'the partners': 'partners', 'liabilities': 'liabilities', 'responsibilities': 'responsibilities', 'a joint venture': 'venture', 'a partnership': 'partnership', 'the two original companies': 'companies', 'their original identity': 'identity'}
OOVs [',']
CONTEXT: [ Foods, that, contain, large amounts, of, fat, ,, al

SUMMARY: [serve]
entity label {}
chunk head {'them': 'them', 'the refrigerator': 'refrigerator', 'these eggs': 'eggs', 'a serving platter': 'platter', 'your guests': 'guests', 'place': 'place'}
OOVs [',']
CONTEXT: [ A good concealer, will, blend, your acne spots, with, the rest, of, your face, so, it, is, much, less, noticeable, ., This, will, prepare, your face, for, the rest, of, the makeup regimen, ., ., Use, a concealer brush, rather, than, your fingers, ., Twist, the brush, over, the pimple, to, get, the concealer, into, all the nooks, and, crannies, !] 

TASK:    [Apply, the concealer, slightly, past, the pimple, to, blend, it, with, the rest, of, your skin, .] 

SUMMARY: [apply, concealer, to, the affected areas]
entity label {}
chunk head {'the concealer': 'concealer', 'the pimple': 'pimple', 'it': 'it', 'the rest': 'rest', 'your skin': 'skin', 'a good concealer': 'concealer', 'your acne spots': 'spots', 'your face': 'face', 'the makeup regimen': 'regimen', 'a concealer brush':

TASK:    [To, turn, this, into, a salad, ,, add, 3 ounces 100 grams, of, crumbled feta cheese, and, a handful, of, pine nuts, .] 

SUMMARY: [go, simple, with, mustard, ,, vinegar, ,, and, olive oil]
entity label {'3 ounces 100 grams': 'QUANTITY', '1 tablespoon': 'QUANTITY'}
chunk head {'a salad': 'salad', '3 ounces 100 grams': 'grams', 'crumbled feta cheese': 'cheese', 'a handful': 'handful', 'pine nuts': 'nuts', 'a bowl': 'bowl', '1 tablespoon': 'tablespoon', 'seeded mustard': 'mustard', '1 tablespoon 15 milliliters': 'milliliters', 'rice wine vinegar': 'vinegar', '2 tablespoons 30 milliliters': 'milliliters', 'extra virgin olive oil': 'oil', 'a dash': 'dash', 'salt': 'salt', 'pepper': 'pepper', 'the dressing': 'dressing', 'over 1 head': 'head', 'broccoli': 'broccoli', 'florets': 'florets'}
OOVs [',']
CONTEXT: [ Communication, is, not, all, about, words, ., Your words, can, be, completely, polite, while, your body language, or, actions, are, communicating, negativity, to, others, ., N

CONTEXT: [ , Combine, a drop, of, mild dish soap, with,  , cup, 60 mL white vinegar, and, 1 quart, .946, L, of, warm water, ., Put, the solution, in, a spray bottle, to, apply, it, most, easily, to, any surface, ., Mix, it, gently, by, rocking, it, back, and, forth, or, stirring, gently, ., Do, nt, shake, it, or, you, ll, create, unwanted suds, ., Dont, worry, if, you, add, a little extra soap, ., You, can, also, use, a combination, of, liquid soap, and, water, .] 

TASK:    [Try, combining, 5 drops, of, soap, to, every quart, .946, L, of, water, .] 

SUMMARY: [prepare, a cleaning solution, with, soap, ,, vinegar, ,, and, water]
entity label {}
chunk head {'5 drops': 'drops', 'soap': 'soap', 'every quart': 'quart', 'l': 'l', 'water': 'water', 'a drop': 'drop', 'mild dish soap': 'soap', 'cup': 'cup', '60 ml white vinegar': 'vinegar', '1 quart': 'quart', 'warm water': 'water', 'the solution': 'solution', 'a spray bottle': 'bottle', 'it': 'it', 'any surface': 'surface', 'you': 'you', 'unw

...

CONTEXT: [ A power, of, attorney, does, not, have, to, go, to, a relative, ., In, fact, ,, you, can, name, anybody, you, wish, to, act, on, your behalf, ,, and, you, can, name, specific topics, for, your representative, to, handle, for, you, ., If, you, wish, ,, for, example, ,, to, have, someone, handle, certain business affairs, while, you, may, be, out, of, the country, ,, you, can, do, that, .] 

TASK:    [In, this case, ,, you, may, wish, to, name, a broker, ,, a financial adviser, ,, or, a trusted friend, who, is, knowledgeable, .] 

SUMMARY: [grant power, of, attorney, to, a, business, associate, or, financial adviser, over, some business matters]
entity label {}
chunk head {'this case': 'case', 'you': 'you', 'a broker': 'broker', 'a financial adviser': 'adviser', 'a trusted friend': 'friend', 'who': 'who', 'a power': 'power', 'attorney': 'attorney', 'a relative': 'relative', 'fact': 'fact', 'anybody': 'anybody', 'your behalf': 'behalf', 'specific topics': 'topics', 'your repres

CONTEXT: [ , Place, your tomatoes, in, the preheated oven, and, let, them, sit, there, for, 30 minutes, .] 

TASK:    [When, ready, ,, the skins, will, show, signs, of, some wrinkling, and, some browning, .] 

SUMMARY: [blister, the tomatoes]
entity label {'30 minutes': 'TIME'}
chunk head {'the skins': 'skins', 'signs': 'signs', 'some wrinkling': 'wrinkling', 'some browning': 'browning', 'your tomatoes': 'tomatoes', 'the preheated oven': 'oven', 'them': 'them', '30 minutes': 'minutes'}
OOVs [',']
CONTEXT: [ This step, is, extremely, important, to, yield, a visually appealing final product, with, the proper texture, and, mouth, feel, ., Melt, your chocolate, over, a double boiler, .] 

TASK:    [To, temper chocolate, :] 

SUMMARY: [temper, your chocolate]
entity label {}
chunk head {'temper chocolate': 'chocolate', 'this step': 'step', 'a visually appealing final product': 'product', 'the proper texture': 'texture', 'mouth': 'mouth', 'your chocolate': 'chocolate', 'a double boiler': 'bo

entity label {'daily': 'DATE', 'two weeks': 'DATE', 'Johnson & Johnsons': 'ORG', 'Aveeno Baby': 'ORG', 'Wash': 'PERSON'}
chunk head {'it': 'it', 'two weeks': 'weeks', 'shampoos': 'shampoos', 'adult eczema': 'eczema', 'your childs delicate skin': 'skin', 'a regular mild baby shampoo': 'shampoo', 'johnson & johnsons': 'johnsons', 'aveeno baby': 'baby', 'your childs hair': 'hair', '1% to 2% ketoconazole shampoo': 'shampoo', 'infants': 'infants', 'you': 'you', 'your pediatrician': 'pediatrician', 'treatment': 'treatment'}
OOVs [',']
CONTEXT: [ Your attorney, may, need, time, to, catch, up, to, speed, ., She, can, file, for, a continuance, at, the same time, that, the motion, for, substitution, of, counsel, is, filed, .] 

TASK:    [Before, granting, it, ,, the judge, will, consider, the reasons, for, the substitution, and, the effect, of, a delay, on, the case, ., Your new attorney, should, also, send, notice, to, all, of, the other parties, in, the litigation, informing, them, that, you, 

entity label {'Google Drive': 'FAC'}
chunk head {'you': 'you', 'your google account': 'account', 'google drive': 'drive', 'your email address': 'address', 'password': 'password', 'it': 'it', 'https://www.drive.google.com/.': 'https://www.drive.google.com/.'}
OOVs ['https://www.drive.google.com/.', ',']
CONTEXT: [ , Describe, your business, or, the objective, of, your page, in, a few words 155 characters, ., Add, any website address, if, you, have, one, ,, and, choose, whether, your page, is, a community page, or, not, ,, and, an, official, or, just, a, fan, page, .] 

TASK:    [Tap Save Info, to, save, the details, .] 

SUMMARY: [modify, your page]
entity label {'Tap Save Info': 'ORG', 'one': 'CARDINAL'}
chunk head {'tap save info': 'info', 'the details': 'details', 'your business': 'business', 'the objective': 'objective', 'your page': 'page', 'a few words 155 characters': 'characters', 'any website address': 'address', 'you': 'you', 'a community page': 'page'}
OOVs [',']
CONTEXT: [ I

entity label {'your iPod': 'ORG'}
chunk head {'you': 'you', 'entire playlists': 'playlists', 'your ipod': 'ipod', 'it': 'it', 'a sidebar': 'sidebar', 'the right': 'right', 'the itunes window': 'window', 'the content': 'content', 'your ipod name': 'name', 'the mouse': 'mouse', 'trackpad': 'trackpad'}
OOVs [',']
CONTEXT: [] 

TASK:    [ Place, a wire baking rack, in, a large roasting pan, .] 

SUMMARY: [preheat, your oven, to, 375f 190c]
entity label {}
chunk head {'a wire baking rack': 'rack', 'a large roasting pan': 'pan'}
OOVs []
CONTEXT: [ , Identify, the date, of, the first full moon, that, comes, after, the vernal equinox, ., This date, will, land, no later than a month, after, the vernal equinox, ., You, can, find, out, this information, by, checking, a lunar calendar, ., These calendars, track, and, illustrate, the, day, -, by, -, day, phases, of, the moon, .] 

TASK:    [You, can, buy, a lunar wall, or, desk calendar, ,, or, you, can, search, for, a, free, one, online, .] 

SUMM

In [11]:
# create vocabulary
vocab = Vocab(os.path.join(config.vocab_path, 'vocab3.txt'));

Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated word in vocabulary file: "
Duplicated w

In [11]:
vocab

In [14]:
# argument namespace
opt = Namespace(
    train_mle = "yes", 
    train_rl = "no", 
    mle_weight = 1., 
    load_model = None,
    new_lr = None, 
    rl_weight = 0.)

# create train and val batchers
wiki_batcher = TaskBatcher(
    examples=wiki_train.to_dict('records')[:2000],
    vocab=vocab,
    mode='train',
    batch_size=16,
    single_pass=False
)
wiki_val_batcher = TaskBatcher( # Batching obj
    examples=wiki_val.to_dict('records')[:200],
    vocab=vocab, 
    mode='train', 
    batch_size=50, 
    single_pass=False
)

In [15]:
# make trainer
wiki_trainer = TaskTrain(vocab, wiki_batcher, opt, TaskModel, wiki_val_batcher)

# load pre-trained embedding weights
wiki_trainer.model.load_embeddings("embedding_43861_200.tar")


In [16]:
# train
mle_losses = wiki_trainer.trainIters(n_iters=10, report_every=1, save_every = 2)

iter: 1 mle_loss: 9.829 mle_loss_val: -100.0000
iter: 2 mle_loss: 9.323 mle_loss_val: 9.5534
model saved at: 
 data/saved_models/0000002.tar
iter: 3 mle_loss: 9.491 mle_loss_val: 9.5534
-------------------Keyboard Interrupt------------------


NameError: name 'exit' is not defined

In [ ]:
mle_losses

## Fine Tune the Model to Task data

In [ ]:
# Retrain namespace
opt_retrain = Namespace(
    train_mle = "yes", 
    train_rl = "no", 
    mle_weight = 1., 
    load_model = '0000015.tar',
    new_lr = None, 
    rl_weight = 0.)

# create train and val batchers for task data
task_batcher = TaskBatcher(
    examples=task_train.to_dict('records'),
    vocab=vocab,
    mode='train',
    batch_size=16,
    single_pass=False
)
task_val_batcher = TaskBatcher( # Batching obj
    examples=task_val.to_dict('records')[:200],
    vocab=vocab, 
    mode='train', 
    batch_size=50, 
    single_pass=False
)

# create (re)trainer
task_trainer = TaskTrain(vocab, task_batcher, opt_retrain, TaskModel)

In [ ]:
config.save_model_path = 'data/saved_models2/'
mle_losses_task = task_trainer.trainIters(n_iters=3, report_every=1, save_every = 3)

In [ ]:
mle_losses_task

## Generating Summaries 

In [ ]:
load_model = os.path.join(config.log_root, "data/saved_models/0000090.tar") # model directory

# new batcher for evaluation
wiki_eval_batcher = TaskBatcher(
    examples=wiki_val.to_dict('records'),
    vocab=vocab, 
    mode='train', 
    batch_size=50,
    single_pass=True)

task_eval_batcher = TaskBatcher(
    examples=task_val.to_dict('records'),
    vocab=vocab, 
    mode='train', 
    batch_size=50,
    single_pass=True)

evaluator = TaskEvaluate(vocab, task_eval_batcher, TaskModel, load_model) # Evaluation object

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# check model parameter count
count_parameters(evaluator.model)

In [ ]:
decoded_sents, ref_sents, task_sents, context_sents = evaluator.evaluate_batch()

In [ ]:
len(ref_sents)

In [ ]:
len(decoded_sents)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
# df = pd.DataFrame({'task': task_sents,'ref': ref_sents, 'decoded': decoded_sents})
df = pd.DataFrame({'task': task_sents, 'ref': ref_sents, 'decoded': decoded_sents})
df.sample(10)